## Westeros Tutorial Part IVa. - Introducing firm capacity

The next tutorial is comprised of three parts. In the first part we will show you how to ensure that in a model setup which doesnt use sub-annual timesteps and therefore does not depict peak load, we still account for sufficient backup capacity from electricity generation plants to maintain reliability through reasonable load and contingency events.

Further information can be found in https://doi.org/10.1016/j.esr.2013.01.001

In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

## Load the scenario with an emission bound and clone to a new scenario 'firm_capacity'

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='emission_bound')
scen = base.clone(model, 'firm_capacity','illustration of firm-capacity formulation', keep_solution=False)
scen.check_out()

## Retrieve parameters to perform subsequent addition of parameters

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Load Factors - Describing the Electricity Sector
### Peak load 
For systems across the U.S., load duration curves are similar from one location to another. Based on load data it was calculated calculated that the peak load is, on average, 1.7 times the average load (a load factor of 59%).8 Coupled with a standard reserve margin of 15–20% of peak load, that means firm capacity requirements for the U.S. should be roughly twice average load.
Therefore, a peak load factor of 2 is introduced for the level:secondary and commodity:electricity, which is level/commodity onto which all power generation technologies deliver.

In [ ]:
# peak_load_factor(node,commodity,level,year,time)
peak_load_factor = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary',       
        'year': model_horizon,
        'time' : 'year',
        'value' : 2,
        'unit' : '???'})
scen.add_par('peak_load_factor', peak_load_factor)

## Rating bins and reliability factors - Describing the powerplant reliability
### Ensuring enough firm capacity
Toward meeting the firm capacity requirement, conventional generating technologies contribute their nameplate generation capacity. 

This therefore implies that for conventional powerplants, a reliability factor of 1 is assumed.

In order to account for the integration challenge at higher shares of variable renewables, their contribution to the capacity requirements, capacity value, declines as the market share of the technology increases.  We therefore assume, that up to a share of 20% of total electricity supply, the capacity of wind_ppl adds by 80% to the firm capacity of the power system (r1). The remaining 80% installed capacity, however  only  5% contribute to the firm capcity (r2). 

In [ ]:
base_reliability = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '???',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# add the ratings as a set 
scen.add_set('rating', ['r1', 'r2'])

# for Wind PPL
rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.2, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.8, rating= 'r1')
scen.add_par('reliability_factor', reliability_factor)

rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.8, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.05, rating= 'r2')
scen.add_par('reliability_factor', reliability_factor)

# for Coal PPL
reliability_factor = make_df(base_reliability, technology= 'coal_ppl', value = 1, rating= 'firm')
scen.add_par('reliability_factor', reliability_factor)

### Commit and solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700) # scenario: 'firm_capacity' (emission_bound scenario with firm capacity)
b = Plots(base, country, firstyear=700) # scenario: 'emission_bound' (without firm capacity)

When comparing the results of the original scenario without the reliability factor ('emission_bound') and the results of our newly modified scenario ('firm_capacity'), for the same carbon price we can observe that the activity of the wind and the coal powerplant is identical, which is what we hoped to achieve.

## Acitivity for scenario: 'emission_bound' (without firm capacity)

In [ ]:
b.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Acitivity for scenario: 'firm_capacity' (emission_bound scenario with firm capacity)

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

The difference between the two scenarios can be seen when comparing the capacity.  Whereas in the 'emission_bound' scenario, coal capacity is quickly phased out, the 'firm_capacity' scenario shows, that there is a substantial amount of capacity required from the coal_ppl in order for the wind_ppl to achieve such high market share levels.

## Capacity for scenario: 'emission_bound' (without firm capacity)

In [ ]:
b.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Capacity for scenario: 'firm_capacity' (emission_bound scenario with firm capacity)

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

This has a substantial impact on the electricity price, which is reflected in the prices for lighting.

## Light prices for scenario: 'emission_bound' (without firm capacity)

In [ ]:
b.plot_prices(subset=['light'], baseyear=True)

## Light prices for scenario: 'firm_capacity' (emission_bound scenario with firm capacity)

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()